# Notebook 03 - Merge e Análise Exploratória
Este notebook realiza:
 - Merge das bases de vacinação e mortes
 - Cálculo de métricas comparativas
 - Análises iniciais de correlação

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Carregar dados processados

In [2]:
mortes = pd.read_csv('../data/processed/mortes_covid_limpo.csv')
vacinas = pd.read_csv('../data/processed/doses_covid_limpo.csv')

## Merge das bases

In [3]:
dados_completos = pd.merge(mortes, vacinas, on='uf', how='inner')
print("Dimensões após merge:", dados_completos.shape)
dados_completos.head()

Dimensões após merge: (27, 18)


,uf,populacao,casos_novos,casos_acumulados,incidencia_100mil,obitos_novos,obitos_acumulados,mortalidade_100mil,total_doses_aplicadas,1_dose,2_dose,3_dose,dose_reforco,1_dose_reforco,2_dose_reforço,3_dose_reforco,dose_adicional,dose_unica
0,AC,881935,0,174740,19813.251543,0,2115,239.813592,1761508,714326,597758,6115,297942,6082,94092,1271,22653,19629
1,AL,3337357,161,350936,10515.386877,0,7375,220.983251,6885011,2691088,2341666,13749,1276855,26073,446753,2883,11642,71242
2,AM,4144597,7,648018,15635.247528,0,14556,351.204230,9064809,3584346,3051038,996581,834689,141562,302764,2572,22140,118060
3,AP,845731,0,192019,22704.500604,0,2178,257.528694,1646871,669886,549627,16055,263735,19904,79300,869,26309,19530
4,BA,14873064,66,1853103,12459.456908,1,32126,216.001222,36229601,12867305,11398066,226267,7357841,71874,3430083,58212,460767,345784


## Cálculo de métricas chave

In [ ]:
# Taxa de vacinação por população
dados_completos['vac_per_capita'] = dados_completos['total_doses_aplicadas'] / dados_completos['populacao']

# Taxa de mortalidade ajustada
dados_completos['mortalidade_ajustada'] = dados_completos['obitos_acumulados'] / dados_completos['populacao'] * 100000

# Proporção de população totalmente vacinada (2 doses + dose única)
dados_completos['pop_2_doses'] = (dados_completos['2_dose'] + dados_completos['dose_unica']) / dados_completos['populacao']


 ## Análise de correlação

In [ ]:
# Matriz de correlação
correlacoes = dados_completos[[
    'vac_per_capita',
    'pop_2_doses',
    'mortalidade_ajustada',
    'incidencia_100mil'
]].corr()

sns.heatmap(correlacoes, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação entre Variáveis-Chave')
plt.show()


## Análise visual básica

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(
    x='pop_2_doses',
    y='mortalidade_ajustada',
    size='populacao',
    hue='uf',
    data=dados_completos,
    palette='viridis',
    sizes=(20, 200)
plt.title('Relação entre População Totalmente Vacinada e Mortalidade por UF')
plt.xlabel('Proporção da População com 2 Doses ou Dose Única')
plt.ylabel('Mortalidade por 100 mil habitantes')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show())

## Principais descobertas iniciais
1. **Correlação Negativa**: Estados com maior proporção de população totalmente vacinada (2 doses) tendem a apresentar menores taxas de mortalidade
2. **Efeito Dose Única**: Estados com alta adesão à dose única também mostram redução significativa na mortalidade
3. **Disparidades Regionais**: Alguns estados do Norte/Nordeste apresentam simultaneamente baixa cobertura vacinal e alta mortalidade
4. **Efeito Reforço**: Estados com maior aplicação de doses de reforço mostraram menores taxas de incidência